In [37]:
# Unified Router caller (same for all tests)
import requests, json

AGENT_URL = "http://127.0.0.1:8000/collect"  # ← change if different

def call_router(intent: str, args: dict | None = None):
    r = requests.post(AGENT_URL, json={"intent": intent, "args": args or {}}, timeout=60)
    r.raise_for_status()
    resp = r.json()
    print("OK:", resp.get("ok"), "| Intent:", resp.get("intent"))
    if not resp.get("ok"):
        print(json.dumps(resp, indent=2))
        raise SystemExit
    # Show which provider actually answered
    src = (resp.get("meta") or {}).get("source") or {}
    print("Provider(s):", src)
    # Return whole response so you can inspect raw as needed
    return resp

def show_first(obj, n=3):
    if isinstance(obj, list):
        for i, x in enumerate(obj[:n], 1):
            print(f"{i:>2}.", (x if isinstance(x, (str,int,float)) else {k:x.get(k) for k in list(x)[:8]}))
    elif isinstance(obj, dict):
        print({k: obj.get(k) for k in list(obj)[:12]})
    else:
        print(repr(obj))

In [38]:
# What it does: Lists soccer leagues; TSDB primary (falls back to AllSports if empty).
# Try editing:
#   name="English Premier League", "Spanish La Liga", "Italian Serie A"
#   country="England" (TSDB) or countryName="Italy" (AS resolver)

resp = call_router("leagues.list", {
    # "name": "English Premier League",
    # "country": "England",          # TSDB filter
    # "countryName": "Italy",        # AllSports resolver (to countryId)
})
data = resp["data"]
print("Keys:", list(data.keys()))
leagues = data.get("leagues") or data.get("result") or []
print("Total leagues:", len(leagues))
show_first(leagues, n=5)

OK: True | Intent: leagues.list
Provider(s): {'primary': 'tsdb', 'fallback': None}
Keys: ['leagues', 'count']
Total leagues: 34
 1. {'idLeague': '4328', 'strLeague': 'English Premier League', 'strSport': 'Soccer', 'strLeagueAlternate': 'Premier League, EPL'}
 2. {'idLeague': '4329', 'strLeague': 'English League Championship', 'strSport': 'Soccer', 'strLeagueAlternate': 'Championship'}
 3. {'idLeague': '4330', 'strLeague': 'Scottish Premier League', 'strSport': 'Soccer', 'strLeagueAlternate': 'Scottish Premiership, SPFL'}
 4. {'idLeague': '4331', 'strLeague': 'German Bundesliga', 'strSport': 'Soccer', 'strLeagueAlternate': 'Bundesliga, Fußball-Bundesliga'}
 5. {'idLeague': '4332', 'strLeague': 'Italian Serie A', 'strSport': 'Soccer', 'strLeagueAlternate': 'Serie A'}


In [39]:
# What it does: Lists seasons for a league; TSDB primary (AllSports fallback returns Leagues raw).
# Edit leagueName or pass leagueId directly.

resp = call_router("seasons.list", {
    # "leagueName": "English Premier League",
    # "leagueName": "Italian Serie A",
    # "leagueId": "4328"  # TSDB EPL id (example)
})
seasons = resp["data"].get("seasons") or resp["data"].get("result") or []
print("Seasons:", len(seasons))
show_first(seasons, n=10)

OK: True | Intent: seasons.list
Provider(s): {'primary': 'tsdb', 'fallback': 'allsports'}
Seasons: 976
 1. {'league_key': 4, 'league_name': 'UEFA Europa League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': 'https://apiv2.allsportsapi.com/logo/logo_leagues/', 'country_logo': None}
 2. {'league_key': 1, 'league_name': 'UEFA European Championship', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'country_logo': None}
 3. {'league_key': 683, 'league_name': 'UEFA Conference League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'country_logo': None}
 4. {'league_key': 3, 'league_name': 'UEFA Champions League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': 'https://apiv2.allsportsapi.com/logo/logo_leagues/3_uefa_champions_league.png', 'country_logo': None}
 5. {'league_key': 633, 'league_name': 'UEFA Nations League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'country_logo': None}
 6. {'league_key

In [40]:
# What it does: League detail; TSDB primary.
resp = call_router("league.get", {
    "leagueName": "Spanish La Liga",
    # "leagueId": "4335",  # TSDB La Liga (example)
})
print("Keys in data:", list(resp["data"].keys()))
show_first(resp["data"].get("league") or resp["data"])

OK: True | Intent: league.get
Provider(s): {'primary': 'tsdb', 'fallback': None}
Keys in data: ['league']
{'idLeague': '4335', 'idSoccerXML': None, 'idAPIfootball': '7351', 'strSport': 'Soccer', 'strLeague': 'Spanish La Liga', 'strLeagueAlternate': 'LaLiga Santander, La Liga', 'intDivision': '0', 'idCup': '0', 'strCurrentSeason': '2025-2026', 'intFormedYear': '1929', 'dateFirstEvent': '2003-01-05', 'strGender': 'Male'}


In [41]:
# What it does: Table/standings for a season; TSDB primary (AllSports Standings fallback).
# Change season to a valid one for that league.
resp = call_router("league.table", {
    "leagueName": "English Premier League",
    "season": "2014-2015",
    # AllSports fallback: "leagueId": "207", "season": "2020/2021"  # Serie A
})
table = resp["data"].get("table") or (resp["data"].get("result") or {}).get("total") or []
print("Rows:", len(table))
show_first(table, n=8)

OK: True | Intent: league.table
Provider(s): {'primary': 'tsdb', 'fallback': None}
Rows: 20
 1. {'idStanding': '244577', 'intRank': '1', 'idTeam': '133610', 'strTeam': 'Chelsea', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/yvwvtu1448813215.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015'}
 2. {'idStanding': '244578', 'intRank': '2', 'idTeam': '133613', 'strTeam': 'Manchester City', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/vwpvry1467462651.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015'}
 3. {'idStanding': '244579', 'intRank': '3', 'idTeam': '133604', 'strTeam': 'Arsenal', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/uyhbfe1612467038.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015'}
 4. {'idStanding': '244580', 'intRank': '4', 'idTeam': '133612', 'strTeam': 'Manchester United', 'strBadge':

In [42]:
# What it does: Teams by league or by teamName; TSDB primary (AS fallback).
resp = call_router("teams.list", {
    "leagueName": "English Premier League",
    # or: "teamName": "Arsenal"
})
teams = resp["data"].get("teams") or resp["data"].get("result") or []
print("Teams:", len(teams))
show_first(teams, n=8)

OK: True | Intent: teams.list
Provider(s): {'primary': 'tsdb', 'fallback': None}
Teams: 20
 1. {'idTeam': '133604', 'idESPN': '359', 'idAPIfootball': '42', 'intLoved': '8', 'strTeam': 'Arsenal', 'strTeamAlternate': 'Arsenal Football Club, AFC, Arsenal FC', 'strTeamShort': 'ARS', 'intFormedYear': '1892'}
 2. {'idTeam': '133601', 'idESPN': '4328', 'idAPIfootball': '66', 'intLoved': None, 'strTeam': 'Aston Villa', 'strTeamAlternate': 'Aston Villa FC', 'strTeamShort': 'AVL', 'intFormedYear': '1874'}
 3. {'idTeam': '134301', 'idESPN': '349', 'idAPIfootball': '35', 'intLoved': '2', 'strTeam': 'Bournemouth', 'strTeamAlternate': 'AFC Bournemouth, Athletic Football Club Bournemouth', 'strTeamShort': 'BOU', 'intFormedYear': '1890'}
 4. {'idTeam': '134355', 'idESPN': '337', 'idAPIfootball': '55', 'intLoved': '1', 'strTeam': 'Brentford', 'strTeamAlternate': 'Brentford Football Club, Brentford FC, BFC', 'strTeamShort': 'BRE', 'intFormedYear': '1889'}
 5. {'idTeam': '133619', 'idESPN': '331', 'idAPI

In [ ]:
# What it does: Team detail; TSDB primary; AS fallback returns Teams result.
resp = call_router("team.get", {
    # "teamName": "Juventus",
    # "teamId": "133676"  # TSDB Juventus (example)
    # AllSports: "teamId": "96"
})
show_first(resp["data"].get("team") or resp["data"])

OK: True | Intent: team.get
Provider(s): {'primary': 'tsdb', 'fallback': 'allsports'}
{'error': '1', 'result': [{'param': 'teamId', 'msg': 'Required parameter missing', 'cod': 201}]}


In [51]:
# What it does: Team equipment/kit images; TSDB-only feature.
resp = call_router("team.equipment", {
    "teamName": "Juventus",
    # "teamId": "133604"   # TSDB Arsenal (example)
})
equip = resp["data"].get("equipment") or []
print("Equipment rows:", len(equip))
show_first(equip, n=6)

OK: True | Intent: team.equipment
Provider(s): {'primary': 'tsdb', 'fallback': None}
Equipment rows: 44
 1. {'idEquipment': '795', 'idTeam': '133676', 'date': '2019-10-09 14:41:44', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/2019-133676-Jersey.png', 'strType': '1st', 'strUsername': 'smudgie'}
 2. {'idEquipment': '1152', 'idTeam': '133676', 'date': '2020-04-15 22:10:38', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/1yctl81586985025.png', 'strType': '2nd', 'strUsername': 'smudgie'}
 3. {'idEquipment': '1153', 'idTeam': '133676', 'date': '2020-04-15 22:10:55', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/newfxj1586985041.png', 'strType': '3rd', 'strUsername': 'smudgie'}
 4. {'idEquipment': '1687', 'idTeam': '133676', 'date': '2020-09-08 14:30:37', 'strSeason': '2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/e

In [ ]:
# What it does: Players; AllSports primary (supports playerName/teamId/leagueId), TSDB fallback if empty.
# Change teamName/teamId as needed. Example AS teamId: Juventus=96
resp = call_router("players.list", {
    # By team:
    # "teamName": "Juventus",
    # "teamId": "96",             # AllSports Juventus
    # Or by player:
    # "playerName": "Lionel Messi",
})
players = resp["data"].get("result") or resp["data"].get("players") or []
print("Players:", len(players))
show_first(players, n=8)

OK: True | Intent: players.list
Provider(s): {'primary': 'allsports', 'fallback': None}
Players: 30
 1. {'player_key': 3012170221, 'player_name': 'Igor Tudor', 'player_number': None, 'player_country': None, 'player_type': 'Coach', 'player_age': None, 'player_match_played': None, 'player_goals': None}
 2. {'player_key': 41841276, 'player_name': 'M. Perin', 'player_number': '1', 'player_country': 'Italy', 'player_type': 'Goalkeepers', 'player_age': '33', 'player_match_played': '', 'player_goals': ''}
 3. {'player_key': 700014087, 'player_name': 'M. Di Gregorio', 'player_number': '29', 'player_country': 'Italy', 'player_type': 'Goalkeepers', 'player_age': '28', 'player_match_played': '', 'player_goals': ''}
 4. {'player_key': 1136721488, 'player_name': 'C. Pinsoglio', 'player_number': '23', 'player_country': 'Italy', 'player_type': 'Goalkeepers', 'player_age': '35', 'player_match_played': '', 'player_goals': ''}
 5. {'player_key': 1979491368, 'player_name': 'D. Rugani', 'player_number': '